In [38]:
from datasets import Dataset,load_dataset
import json
datasets = []
with open(
    "/data3/yss/sft_datas/0912/sft_data_merge_v18_quality_filtered_sharegpt.jsonl",
    mode="r",
) as f:
    for line in f:
        datasets.append(json.loads(line))
print(len(datasets))

In [60]:
datas = []
for data in datasets[0:10000]:
    data_new = {}
    if "tools" in data:
        if isinstance(data["tools"],list):
            if len(data["tools"]):
                data["tools"] = json.dumps(data["tools"])
            else:
                data["tools"] = ""
        elif data["tools"] =="[]" or data["tools"] =="[ ]" or data["tools"] ==" ":
            data["tools"] = ""
        else:
            data["tools"] = data["tools"]
    else:
        data["tools"] = ""

    data_new["tools"] = data["tools"]
    data_new["system"] = data["system"]
    data_new["conversations"] = data["conversations"]
    datas.append(data_new)
    
# data_path_trans = "/data3/yss/sft_datas/0912/sft_data_merge_v18_quality_filtered_sharegpt_trans.json"
data_path_trans = "./sft_data_merge_v18_quality_filtered_sharegpt_trans.json"
# save_jsonl(datas, data_path_trans)
with open(data_path_trans,"w")as f:
    json.dump(datas,f,ensure_ascii=False)

dataset = load_dataset(
    "json",
    data_files=data_path_trans,
    split="train",
)
print(dataset)

Generating train split: 10000 examples [00:00, 12567.20 examples/s]

Dataset({
    features: ['tools', 'system', 'conversations'],
    num_rows: 10000
})


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/data0/pretrained-models/Qwen2-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)

prompt = "Give me a short introduction to large language model."
messages = [
    # {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
text


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>human\nGive me a short introduction to large language model.<|im_end|>\n<|im_start|>assistant\n'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
print(model)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=512)
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)